In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header = None, engine='python', encoding = 'latin-1')

In [4]:
user = pd.read_csv('ml-1m/users.dat', sep='::', header = None, engine='python', encoding = 'latin-1')

In [5]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header = None, engine='python', encoding = 'latin-1')

In [12]:
# Creating training and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [20]:
# Obtaining the total number of users and movies to create 2d matrix that contains the movies corresponding to each user

# Largest id for customer and movie might be on training or test set, thus this verification
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [24]:
# Converting 2d array where x axis = users (observations) and y axis = movies (features), which is what the RBM expects
def convert(data):
#     Using lists for pytorch
    new_data = []
    for id_users in range(1,nb_users + 1):
#         Getting all the movies and their ratings per user, and adding it to new_data
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
#         If user did not watch movie, rating must be 0
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [25]:
training_set = convert(training_set)
test_set = convert(test_set)

In [34]:
# Converting lists to torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [35]:
# To predict whether a customer liked or not a movie we need to convert ratings into binary classfiers and mantain consistency with input/output of model
# 1 == liked, 0 == disliked
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [39]:
# Creating model for RBM
class RBM():
    def __init__(self, nv, nh):
#         Initializing tensor of size nh nv filled with random #s from a normal distribution with mean 0 and variance 1 (also called the standard normal distribution)
        self.W = torch.randn(nh, nv)
#     Initializing bias for probabilities for visible nodes given a hidden node and for probabilities of hidden nodes given a visible node
#     We need it to add a col to add element corresponding to batch size (following pytorch sintax)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

#   Where x is vn
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
#         Making sure that bias is included in each line of the batch
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
#         Returning list of probabilities of hs being activated by v, and binary outcome of each hs being activated (bernoulli sampling)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

#   For Gibbs sampling and predictions
#   Where y is hn
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
#   For contrastive divergence
#   v0  == input data (ratings), vk == vn visible after k-samplings, ph0 == probabilites 1st iter, phk == probabilities of h after k iter
    def train(self, v0, vk, ph0, pk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
#         To mantain 2d
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [40]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [43]:
# Training model
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
#         Doing the k steps of CD
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
#             To ensure that -1 ratings (not watched) mantain the same values
            vk[v0 < 0] = v0[v0 < 0]
        phk,_= rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
#         Ensuring that -1 ratings (not watched) are not used to train the model
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('Epoch: ' + str(epoch) + ' Loss: ' + str(train_loss / s))

Epoch: 1 Loss: tensor(0.2502)
Epoch: 2 Loss: tensor(0.2469)
Epoch: 3 Loss: tensor(0.2460)
Epoch: 4 Loss: tensor(0.2492)
Epoch: 5 Loss: tensor(0.2452)
Epoch: 6 Loss: tensor(0.2461)
Epoch: 7 Loss: tensor(0.2469)
Epoch: 8 Loss: tensor(0.2447)
Epoch: 9 Loss: tensor(0.2448)
Epoch: 10 Loss: tensor(0.2486)


In [44]:
# Testing RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user : id_user + 1]
    vt = test_set[id_user : id_user + 1]
#     If the user left a rating, predict what it was
    if len(vt[vt >= 0] > 0):
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        train_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        s += 1.
print('Test loss: ' + str(test_loss / s))

Test loss: 0.0


tensor([[-1., -1., -1.,  ..., -1., -1., -1.]])